In [1]:
import os
import glob
from utils import Resume_Evaluator

In [ ]:
resume_evaluator=Resume_Evaluator('azure_openai')

In [ ]:
final_sort=resume_evaluator.evaluate('Document10.pdf', 'resumes')

In [ ]:
final_sort

{'sashidhar_GenAI.pdf': [{'strengths': ['2+ years of experience in implementing and deploying machine learning and deep learning frameworks on cloud platforms like AWS and GCP',
    '1+ years of professional experience with NLP and Large Language Models (LLM)',
    'Proficiency in Python programming, with working experience over one of the ML Frameworks - SKLearn, XGBoost, PyCaret, and Deep Learning Frameworks - Tensorflow, PyTorch, Huggingface, and Rasa'],
   'gaps': ['Bachelor’s Degree (might be final course student) in Statistics, Applied Mathematics, Computer Science, or other related fields',
    'LLM Model Fine-Tuning: Utilize your expertise in encoding and language understanding to fine-tune existing Large Language Models for optimal performance in solving domain-specific needs.',
    'Model Training and Evaluation: Conduct rigorous testing and evaluation of LLMs to identify areas for improvement and implement solutions to enhance their performance.'],
   'questions_to_candidate

In [6]:
import json
import time
import PyPDF2
from openai import AzureOpenAI

In [7]:
gpt_engine_name="gpt-35-turbo-16k"
embedding_model_name="text-embedding-002"
azure_endpoint = "https://securityappdewa.openai.azure.com"
api_key="576b185ff87d4f50962560d349d06332"
api_version= "2023-07-01-preview"

In [8]:
openai_client = AzureOpenAI(
                azure_endpoint = azure_endpoint,
                api_key=api_key,
                api_version= api_version
            )

In [9]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text() if page.extract_text() else ""
    return text

In [2]:
import os
l=os.listdir("resumes")

In [4]:
for i in l:
    if i=="sashidhar_GenAI.pdf":
        print("resumes"+"/"+"sashidhar_GenAI.pdf")

resumes/sashidhar_GenAI.pdf


In [10]:
resume_text=extract_text_from_pdf('resumes\RaviKumarCV.pdf')

In [11]:
job_description=extract_text_from_pdf('Document10.pdf')

In [69]:
def extract_agent(jd):
        res = openai_client.chat.completions.create(model=gpt_engine_name,
                                                        messages=[
                                                                {'role': 'system','content':f'''As a job recruiter agent, your task is to extract the required details mentioned in the given job description and output them as a JSON response.
                                                                
Step 1: Analyze the job description provided below to extract precise details.
Job Description:
```{jd}``` 

Step 2: Based on the analysis in step 1, extract the 'educational degree' that a candidate must have as mentioned in the job description. Provide the precise education details specified. If there are multiple optional degrees, list them in the same string separated by "or." parameter. Output it as a list. If the educational requirement is not mentioned in the job description, provide an empty list.                                                                                                                                                                                                                                 
                                                        
Step 3: Based on the analysis in step 1, extract the both technical, soft 'skills' that the recruiter is seeking in a candidate from the 'job description'. List each skill 'individually and distinctly', without grouping multiple skills together. Exhaustively extract all the skills mentioned in the job description as 'individual entities'.

Step 4: Based on the analysis in step 1, extract the level of experience that the recruiter is looking for in a candidate from the job description. List the experience level(s) in a list. If the experience level is not mentioned in the job description, then it shoukd be empty list.
The skills are very important, so please extract them exhaustively, including those mentioned implicitly.

Step 5: Based on the analysis in step 1, extract whether the recruiter is looking for a candidate with experience in a specific industry or possessing particular domain knowledge. If no such details are mentioned in the job description, provide an empty list.

Step 5: Based on the analysis in step 1, Extract all other requirements mentioned in the job description that do not fall under the categories of Educational Qualifications, Skills, Experience Level, or Industry and Domain Knowledge. Provide them in a list.

Step 6: Output a JSON Response with the following keys: "Educational Qualifications," "Skills," "Experience Level,", "Industry and Domain Knowledge." and "others". The values for each key should be lists of the respective extracted details. Ensure that the extracted values are not repeated across different lists.

Extract as many requirements as possible from the given job description into their respective categories.
'''},  
                                                                {'role': 'user',  'content':'''Based on the give above instructions extract the requirements from the job descripion. When extracting the skills, 'ensure each skill is listed as an individual entity' in the list. "If the requirements are separated by 'or, /' indicating that any one of them is acceptable to the recruiter, list them as a single entity in a string.". Extract as many requirements as possible.
Be precise and thoroughly extract all the values from the job description. Output the Json Response with keys and list of values. Be precise with the requirements; avoid providing any vague or generic details.
Json Response:'''}]
                                                        )
        requirements=json.loads(res.choices[0].message.content)

        return requirements

In [70]:
requirements=extract_agent(job_description)

In [76]:
job_requires=[]
for i, j in requirements.items():
    job_requires.extend(j)

In [31]:
def recruiter_agent(messages):
    res = openai_client.chat.completions.create(model=gpt_engine_name,
                                                messages=messages,
                                                temperature=0)
    return res.choices[0].message.content

In [479]:
def candidate_agent(messages):
    res = openai_client.chat.completions.create(model=gpt_engine_name,
                                                messages=messages,
                                                max_tokens=200,
                                                temperature=0)
    return res.choices[0].message.content

In [451]:
def evaluator_agent(messages):
    res = openai_client.chat.completions.create(model=gpt_engine_name,
                                                messages=messages,
                                                temperature=0)
    score=json.loads(res.choices[0].message.content)
    return score['score']

In [465]:
recruiter_system='''Your are a recruiter look for a best candidate for the vacant position. Your task is to ask the candidate question based on the given requirement below for job.

Job Requirement:
```
{job_requirement}
```
The above is the job requirement for the vacant posiiton, so please ask the candidate a question on this requirement. Ask the question directly without introducing yourself or adding extra information.
'''

In [471]:
candidate_system=f'''You are a candidate looking for a job. You only have the expertise, skills, and experience mentioned in the resume below. Apart from the information provided in the resume, you have no additional knowledge or experience. Your resume is mentioned below in triple backticks(```).

Step 1: Analyse your resume given below.
Resume:
```{resume_text}```

Step 2: Based on the analysis in Step 1, you should respond to the recruiter's questions solely using the information in your resume. Analyze the recruiter's questions carefully and base your answers strictly on the details provided in your resume.
You 'should not lie' about anything to the recruiter, as lying will severely impact your chances of being selected for the job. If you lie, you will be disqualified from the recruitment process. If you do not know answer to the recruiter question then simply say "sorry, I do not know answer to your question".
When responding to the recruiter's questions, be precise and answer based solely on the information provided in the 'resume'. Avoid making "assumptions or adding any information that isn't explicitly mentioned in the resume" in your answer

Step 3: Ensure that the answer is precise and 'based on the resume'. Ensure your answer is strictly under 200 words. Base your response solely on information from the resume, without including any external knowledge.
Your answer should be "precise", using only the terms and wording specified in the resume. "Do not include anything not mentioned in the resume when communicating with the recruiter".

Step 4: Ensure that the "answer is exclusively based on the resume", with 'every word sourced directly' from it. The answer should focus solely on the topic asked by the recruiter, without discussing anything else.

Follow the above steps and give "precise answer", to the point answer to recruiter question.
'''

In [444]:
candidate='''In my role as a Senior Data Scientist at Celebal Technologies, I have gained extensive experience in supporting all stages of the ML model life cycle. I have worked on projects involving AI solutions, prompt optimization, and prompt engineering using LLM models and Python programming.

During the development phase, I have been responsible for robust data extraction from various file types, such as PDFs, Word documents, Excel spreadsheets, and more. I have utilized entity recognition, summarization, and question-answering techniques using Azure OpenAI LLM models. Additionally, I have applied prompt engineering methodologies like CoT and ReACT, along with frameworks like CO-STAR, to ensure accuracy and efficiency.

In the deployment phase, I have contributed to the enhancement of LangChain LLM Agents, leveraging advanced NLP techniques for language understanding and generation. I have also ensured the accuracy of generated responses by leveraging the RAGAS Library.

Throughout the ML model life cycle, I have focused on optimizing performance and user experiences. I have implemented security measures, integrated notification systems, and introduced interactive features to enhance user engagement. Furthermore, I have collaborated closely with stakeholders to gather requirements and deliver context-aware answers.
'''

In [483]:
evaluator_system=f'''You are an job evaluator agent. You will be given job requirement and candidate answer for the requirement. Your task is to analyze the candidate's answer based on the job requirement and determine how well it meets them. Assign a score between 0 and 1 to the candidate's answer, where 0 means no match, 1 means a perfect match, and any value between 0 and 1 indicates a partial match.
Do not be generic in your evaluation. Thoroughly analyze how the candidate's answer meets the job requirements and provide the most accurate score. Only award a full score if the candidate's answer completely satisfies the job requirements in every aspect. Do not give a full score easily.
If the job requirement is experience with the project life cycle, only assign full marks if the candidate's answer covers all aspects of the project life cycle—requirement gathering, planning, design, development, testing, deployment, and maintenance. If only few are mentioned, then assign "partial marks".

Step 1: Analyse the given job requirement and candidate answer.
job requirement: 
```{job_requiremet}```

Candidate answer: 
```{candidate}```

Step 3: Verify whether the candidate's response to the job requirement is accurate and aligns with the job requirements. If the candidate's answer thoroughly and explicitly satisfies all the job requirements, assign a score of 1. If the candidate's response does not meet the job requirements, assign a score of 0.
The score ranges from 0 to 1. If the candidate's answer only partially matches the job requirements, assign a 'score between 0 and 1' based on the level of matching. "only in rare cases where answer matches all requirements you should assign a score of 1". You should assign 'partial marks' in the case of partial requirements matching.
If the candidate states they do not have expertise in the job requirement, assign a score of 0.

Step 4: Output a Json Response with score as key and respective score as value. Do not output anything extra information, just give score.'''

In [473]:
print(evaluator_system)

You are an job evaluator agent. You will be given job requirement and candidate answer for the requirement. Your task is to analyze the candidate's answer based on the job requirement and determine how well it meets them. Assign a score between 0 and 1 to the candidate's answer, where 0 means no match, 1 means a perfect match, and any value between 0 and 1 indicates a partial match.
Do not be generic in your evaluation. Thoroughly analyze how the candidate's answer meets the job requirements and provide the most accurate score. Only award a full score if the candidate's answer completely satisfies the job requirements in every aspect. Do not give a full score easily.
If the job requirement is experience with the project life cycle, only assign full marks if the candidate's answer covers all aspects of the project life cycle—requirement gathering, planning, design, development, testing, deployment, and maintenance. If only few are mentioned, then assign "partial marks".

Step 1: Analyse

In [474]:
evaluator_agent([{'role': 'system','content':evaluator_system}])

1

In [475]:
recriter_messages=[{'role': 'system','content':recruiter_system}]
candidate_messages=[{'role': 'system','content':candidate_system}]
evaluation_messages=[{'role': 'system','content':evaluator_system}]

In [484]:
def cal_score(job_requires, resume_text):
    QnA=[]
    scores=0
    for i in job_requires:
        recruiter_system='''Your are a recruiter look for a best candidate for the vacant position. Your task is to ask the candidate question based on the given requirement below for job.

Job Requirement:
```
{job_requirement}
```
The above is the job requirement for the vacant posiiton, so please ask the candidate a question on this requirement. Ask the question directly without introducing yourself or adding extra information.
'''
        evaluator_system='''You are an job evaluator agent. You will be given job requirement and candidate answer for the requirement. Your task is to analyze the candidate's answer based on the job requirement and determine how well it meets them. Assign a score between 0 and 1 to the candidate's answer, where 0 means no match, 1 means a perfect match, and any value between 0 and 1 indicates a partial match.
Do not be generic in your evaluation. Thoroughly analyze how the candidate's answer meets the job requirements and provide the most accurate score. Only award a full score if the candidate's answer completely satisfies the job requirements in every aspect. Do not give a full score easily.
If the job requirement is experience with the project life cycle, only assign full marks if the candidate's answer covers all aspects of the project life cycle—requirement gathering, planning, design, development, testing, deployment, and maintenance. If only few are mentioned, then assign "partial marks".

Step 1: Analyse the given job requirement and candidate answer.
job requirement: 
```{job_requiremet}```

Candidate answer: 
```{candidate}```

Step 3: Verify whether the candidate's response to the job requirement is accurate and aligns with the job requirements. If the candidate's answer thoroughly and explicitly satisfies all the job requirements, assign a score of 1. If the candidate's response does not meet the job requirements, assign a score of 0.
The score ranges from 0 to 1. If the candidate's answer only partially matches the job requirements, assign a 'score between 0 and 1' based on the level of matching. "only in rare cases where answer matches all requirements you should assign a score of 1". You should assign 'partial marks' in the case of partial requirements matching.
If the candidate states they do not have expertise in the job requirement, assign a score of 0.

Step 4: Output a Json Response with score as key and respective score as value. Do not output anything extra information, just give score.
'''
        recruiter_system=recruiter_system.format(job_requirement=i)
        recriter_messages[0]={'role': 'system','content':recruiter_system}
        recruiter=recruiter_agent(recriter_messages)
        candidate_messages.append({'role': 'user',  'content':recruiter})
        candidate= candidate_agent(candidate_messages)

        evaluator_system=evaluator_system.format(job_requiremet=i, candidate=candidate)
        evaluation_messages[0]={'role': 'system','content':evaluator_system}
        score=evaluator_agent(evaluation_messages)
        scores+=score
        QnA.append({'recruiter':recruiter, 'candidate':candidate, 'score':score})

        print("Recruiter: ", recruiter)
        print("Candidate: ", candidate)
        print(score)
        print('-------------------------------------------------------------------------------------------------------------------')
        candidate_messages.pop(1)

    return QnA, scores

In [485]:
q, s=cal_score(job_requires, resume_text)

Recruiter:  Do you have a Bachelor's Degree in Statistics?
Candidate:  No, I do not have a Bachelor's Degree in Statistics. My educational background includes a B.Sc. in Physics, Chemistry, and Mathematics from the University of Lucknow.
0
-------------------------------------------------------------------------------------------------------------------
Recruiter:  Do you have a Bachelor's Degree in Applied Mathematics?
Candidate:  No, I do not have a Bachelor's Degree in Applied Mathematics. My educational background includes a B.Sc. in Physics, Chemistry, and Mathematics from the University of Lucknow.
0
-------------------------------------------------------------------------------------------------------------------
Recruiter:  Do you have a Bachelor's Degree in Computer Science?
Candidate:  No, I do not have a Bachelor's Degree in Computer Science. My educational background includes a B.Sc. in Physics, Chemistry, and Mathematics from the University of Lucknow.
0
------------------

In [393]:
s/len(job_requires)

0.8040540540540541

In [486]:
s/len(job_requires)

0.5945945945945947

In [497]:
strength=""
gaps=""
questions=""
for i in q:
    if i['score']==0:
        gaps+=i['recruiter']+'\n'
    elif i['score']==1:
        strength+=i['recruiter']+'\n'
    else:
        questions+=i['recruiter']+'\n'

In [490]:
def find_gaps(messages):
    res = openai_client.chat.completions.create(model=gpt_engine_name,
                                                messages=messages,
                                                temperature=0)
    res=res.choices[0].message.content
    return res

In [510]:
message=[{'role': 'system','content':f'''You will be provided with some points indicating the candidate's weaknesses. Based on these points, rephrase and list upto 5 most significant weaknesses. List them in a list and give a Json Response with weakness as key.
weak points:
{gaps}

Output the Json Response with weakness as key and list of weaknesses as values.
          
Json Response:'''}]

In [ ]:
message=[{'role': 'system','content':f'''You will be provided with some points indicating the candidate's strengths. Based on these points, rephrase and list upto 5 most significant strengths. List them in a list and give a Json Response with strength as key.
weak points:
{strength}

Output the Json Response with strength as key and list of strengths as values.
          
Json Response:'''}]

In [517]:
message=[{'role': 'system','content':f'''You will be provided with some points indicating the candidate's gaps. Based on these points, rephrase and list upto 5 questions that need to be asked to candidate to clarify them. List them in a list and give a Json Response with questions as key.
weak points:
{questions}

Output the Json Response with strength as key and list of strengths as values.
          
Json Response:'''}]

In [518]:
aq=find_gaps(message)

In [519]:
aq

'{\n  "questions": [\n    "What experience do you have in Natural Language Processing (NLP)?",\n    "Do you have experience working with large language models?",\n    "Do you have experience working with Google Cloud Platform (GCP)?",\n    "Can you explain the process of model fine-tuning?",\n    "Can you describe your experience in developing CI/CD pipelines?"\n  ]\n}'

In [527]:
st=""
for i, j in json.loads(aq).items():
    for k in j:
        st+=k+'\n'

In [528]:
print(st)

What experience do you have in Natural Language Processing (NLP)?
Do you have experience working with large language models?
Do you have experience working with Google Cloud Platform (GCP)?
Can you explain the process of model fine-tuning?
Can you describe your experience in developing CI/CD pipelines?



In [529]:
# Sample dictionary with scores
scores = {
    'item1': 0.9,
    'item2': 0.5,
    'item3': 0.75,
    'item4': 1.0
}

# Sort the dictionary by values
sorted_scores = dict(sorted(scores.items(), key=lambda item: item[1], reverse=True))

# Print the sorted dictionary
print(sorted_scores)

{'item4': 1.0, 'item1': 0.9, 'item3': 0.75, 'item2': 0.5}


In [530]:
for i, j in sorted_scores.items():
    print(i)

item4
item1
item3
item2


### langgraph

In [113]:
%pip install -U langgraph

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.20 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.28 which is incompatible.
langchain-openai 0.1.6 requires langchain-core<0.2.0,>=0.1.46, but you have langchain-core 0.2.28 which is incompatible.
langchain-experimental 0.0.58 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.28 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.28 which is incompatible.
You should consider upgrading via the 'c:\Users\DudukuntaSashidharRe\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.



  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.67
    Uninstalling langsmith-0.1.67:
      Successfully uninstalled langsmith-0.1.67
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52


In [299]:
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import AzureChatOpenAI
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

In [358]:
# Define the tools for the agent to use
@tool
def user_approved_to_send_mail(email_content: str):
    """when user approves to send mai after asking for review
     Input: content of the mail"""
    # This is a placeholder, but don't tell the LLM that...
    print("Email sent")
    return "Email successfully sent"

In [359]:
# def write_email(state: MessagesState):
#     """Writes the email content ."""
#     messages = state['messages']
#     llm = AzureChatOpenAI(
#     api_key="576b185ff87d4f50962560d349d06332",
#     azure_endpoint="https://securityappdewa.openai.azure.com/",
#     openai_api_version="2023-07-01-preview",
#     azure_deployment="gpt-35-turbo-16k",
#     temperature=0
# )
#     response = llm.invoke(messages)

#     print("response", response)
#     # We return a list, because this will get added to the existing list
#     return {"messages": [response]}

In [360]:
def take_human_input(state: MessagesState):
    "Taking input from human"
    inp=input()
    return {"messages": [HumanMessage(content=inp)]}

In [361]:
tools = [user_approved_to_send_mail, take_human_input]

tool_node = ToolNode(tools)

model = model = AzureChatOpenAI(
    api_key="576b185ff87d4f50962560d349d06332",
    azure_endpoint="https://securityappdewa.openai.azure.com/",
    openai_api_version="2023-07-01-preview",
    azure_deployment="gpt-35-turbo-16k",
    temperature=0
).bind_tools(tools)

In [362]:
# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    print("tools", last_message.tool_calls)
    if last_message.tool_calls:
        if last_message.tool_calls[0]['name']=="user_approved_to_send_mail":
            print("in tools")
            return "take_human_input"
        # Otherwise, we stop (reply to the user)
        else:
            return "take_human_input"
    else:
        return "human_loop"

In [363]:
# Define the function that calls the model
def call_model(state: MessagesState):
    messages = state['messages']
    print('message', messages)
    response = model.invoke(messages)
    print("response", response)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [372]:
# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_node("human_loop", take_human_input)

In [373]:
# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

In [374]:
# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

In [377]:
# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('agent', "human_loop")
workflow.add_edge('agent', "tools")

# Initialize memory to persist state between graph runs
checkpointer = MemorySaver()

In [378]:
app = workflow.compile(checkpointer=checkpointer)

In [379]:
# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="write a mail to sashidhar for schedule interview")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

message [HumanMessage(content='write a mail to sashidhar for schedule interview', id='348f2264-6d24-4d46-934b-add129c21c9b')]
response content='Sure, I can help you with that. Could you please provide me with the details of the interview, such as the date, time, and location?' response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 375, 'total_tokens': 407}, 'model_name': 'gpt-35-turbo-16k', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe

KeyError: 'human_loop'

In [228]:
# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="hi hello")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

message [HumanMessage(content='write a mail to sashidhar for schedule interview', id='a4c32c83-e70a-4e25-a095-e087c08ec884'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mcGAR2VTgeXmAIsiW1cgYaE2', 'function': {'arguments': '{\n  "email_content": "Dear Sashidhar,\\n\\nI hope this email finds you well. I am writing to schedule an interview with you for the position you have applied for at our company.\\n\\nThe interview will be conducted on [date] at [time]. The interview will take place at our office located at [address].\\n\\nPlease confirm your availability for the interview by replying to this email or contacting me at [phone number].\\n\\nIf you have any questions or need any further information, please feel free to reach out to me.\\n\\nWe look forward to meeting you and discussing your qualifications further.\\n\\nBest regards,\\n[Your Name]\\n[Your Position]\\n[Company Name]"\n}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage':

'Hello! How can I assist you today?'

In [156]:
print(final_state["messages"][-1].content)

Subject: Invitation for Interview - [Company Name]

Dear Sashidhar,

I hope this email finds you well. I am writing to invite you for an interview for the position of [Job Title] at [Company Name]. We were impressed with your qualifications and experience, and we believe that you would be a valuable addition to our team.

The interview will be conducted on [Date] at [Time]. The interview will take place at our office located at [Address]. Please make sure to arrive at least 10 minutes before the scheduled time.

During the interview, we will discuss your skills, experience, and how they align with the requirements of the position. We will also provide you with more information about our company and the role you are applying for. The interview process may include a combination of technical assessments, behavioral questions, and a discussion about your career goals.

To confirm your availability and schedule the interview, please reply to this email with your preferred date and time. If 

In [277]:
while True:
    inp=input()
    # Use the Runnable
    final_state = app.invoke(
        {"messages": [SystemMessage(content="Your task is to write a mail to candidate and ask for the review, if user says this mail looks good and asks to send then only send the mail."),
            HumanMessage(content=inp)]},
        config={"configurable": {"thread_id": 42}}
    )

    print(final_state["messages"][-1].content)
    if inp=='send':
        break
    print("Does this email looks good? do you want to add anything?")

message [HumanMessage(content='write a mail to sashidhar for schedule interview', id='5408552f-2b4e-46e0-851d-1324891bc437'), AIMessage(content='Subject: Interview Scheduling Request\n\nDear Sashidhar,\n\nI hope this email finds you well. I am reaching out to you on behalf of our company to schedule an interview for a position you have applied for.\n\nWe have reviewed your application and are impressed with your qualifications and experience. We believe you could be a great fit for the role and would like to invite you for an interview to further discuss your skills and potential contributions to our team.\n\nThe interview will be conducted on [date] at [time] via [video call/phone call/in-person]. Please let us know your availability during the following time slots:\n\n1. [Date and time]\n2. [Date and time]\n3. [Date and time]\n\nIf none of the above time slots work for you, please suggest an alternative time and we will do our best to accommodate it.\n\nTo confirm your availability o

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_bfpfhbihnZkmCe1FLk2gbu8Z", 'type': 'invalid_request_error', 'param': 'messages.[2].role', 'code': None}}

In [1]:
import json
import time
import PyPDF2
from openai import OpenAI
from openai import AzureOpenAI
from config import api_key, azure_endpoint, api_version, gpt_engine_name

In [2]:
class Email_Agent:
    def __init__(self, openai_type):
        if openai_type=="azure_openai":
            self.openai_client = AzureOpenAI(
                    azure_endpoint = azure_endpoint,
                    api_key=api_key,
                    api_version= api_version
                )
        else:
            self.openai_client = OpenAI(api_key=api_key)

        self.messages=[]

    def _email_prompt(self, email, name, company_name, job_title):
        email_system=f'''Your task is to Write email to schedule an interview upon shortlist. Write a precise and short email based on the provided details. If user wants to add extra information then re write the email.
The interview will be in online mode. Do not mention any date or timing in the email or any company contact number. In the closing of the email, include only the company name and omit any recruiter names.
Details to write maile are below:
Email: {email}
Nmae: {name}
company_name: {company_name}
job_title: {job_title}'''
        return email_system
    
    def _find_details(self, resume_text, job_description):
        system_prompt=f'''Your are a resume parser. Your task is to extract the email id, name of candidate from the given resume and company name, job title from the given job description. Output the Json Response with name, email, company_name and job_title as keys and there respective values.

Resume:
```{resume_text}```

Extract the candidate name, email id from the above resume and give it in name, email keys.

Job Description:
```{job_description}```

Extract the company name, job title from the above job description and give it in company_name, job_title keys.

Output a Json Response with name, email, company_name and job_title as keys and there respective values.
Json Response:
'''
        messages=[{'role':'system', 'content':system_prompt}]
        res = self.openai_client.chat.completions.create(model=gpt_engine_name,
                                                    messages=messages,
                                                    temperature=0)
        res=json.loads(res.choices[0].message.content)
        email=res['email']
        name=res['name']
        company_name=res['company_name']
        job_title=res['job_title']
        return email, name, company_name, job_title
    
    def agent(self, resume_text, job_description, user_message):
        if user_message:
            self.messages.append({'role':'user', 'content':user_message})
        else:
            email, name, company_name, job_title=self._find_details(resume_text, job_description)
            email_system=self._email_prompt(email, name, company_name, job_title)
            self.messages.append({'role':'system', 'content': email_system})
        if user_message=="send":
            return "Email has been sucessfully send to the user" 
        res = self.openai_client.chat.completions.create(model=gpt_engine_name,
                                                    messages=self.messages,
                                                    temperature=0)
        result=res.choices[0].message.content                                         
        self.messages.append({'role':'assistant', 'content':result})  

        return result


In [14]:
email_writer=Email_Agent("azure_openai")

In [15]:
e=email_writer.agent(resume_text, job_description, '')
print(e)
print('-------------------------')
print('\nDo you Like to add anything to the mail? If yes, please give us your input')
print("\nIf you feel the email looks good, type 'send' to send it to the candidate.\n\n")
while True:
    user_message=input()
    e=email_writer.agent(resume_text, job_description, user_message)
    print(e)
    print('-------------------------')
    if user_message=="send":
        break
    print('\nDo you Like to add anything to the mail? If yes, please give us your input')
    print("\nIf you feel the email looks good, type 'send' to send it to the candidate.\n\n")

Dear Ravi Kumar,

We are pleased to inform you that you have been shortlisted for the position of LLM Engineer at Fynd. Congratulations on making it to the next round!

We would like to schedule an interview with you to further discuss your qualifications and experience. The interview will be conducted online. Please let us know your availability for the interview so that we can arrange a suitable time.

Looking forward to hearing from you soon.

Best regards,
Fynd
-------------------------

Do you Like to add anything to the mail? If yes, please give us your input

If you feel the email looks good, type 'send' to send it to the candidate.


Dear Ravi Kumar,

We are pleased to inform you that you have been shortlisted for the position of LLM Engineer at Fynd. Congratulations on making it to the next round!

We would like to schedule an interview with you to further discuss your qualifications and experience. The interview will be conducted online. We have scheduled the interview for 6t